In [1]:
from dask.distributed import Client, progress
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, visualize
import numpy as np
import stratify


In [ ]:
client = Client(processes=False, threads_per_worker=2,
                n_workers=1)
client


In [2]:
def src_data(shape=(400, 500, 100)):
    z = np.tile(np.linspace(0, 100, shape[-1]),
                np.prod(shape[:-1])).reshape(shape)
    fz = np.arange(np.prod(shape)).reshape(shape)
    return z, fz


def interp(tgt, z, fz,
           interp=stratify.INTERPOLATE_LINEAR,
            extrap=stratify.EXTRAPOLATE_NAN):
    return stratify.interpolate(tgt, z, fz,
                                interpolation=interp, extrapolation=extrap)

shape=(3000, 1000, 22)
print(np.prod(shape) * 8 / 1024**3)
z, fz = src_data(shape)

fz_lazy = da.asarray(fz)
z_lazy = da.asarray(z)
print(fz_lazy.chunks)


0.49173831939697266
((750, 750, 750, 750), (500, 500), (22,))


In [12]:
tgt_single = np.array([0])
print(tgt_single)

[0]


In [10]:
tgt_many = np.linspace(-20, 120, 50) #* np.ones(shape[:-1] + (50,))
print(tgt_many)

[-20.         -17.14285714 -14.28571429 -11.42857143  -8.57142857
  -5.71428571  -2.85714286   0.           2.85714286   5.71428571
   8.57142857  11.42857143  14.28571429  17.14285714  20.
  22.85714286  25.71428571  28.57142857  31.42857143  34.28571429
  37.14285714  40.          42.85714286  45.71428571  48.57142857
  51.42857143  54.28571429  57.14285714  60.          62.85714286
  65.71428571  68.57142857  71.42857143  74.28571429  77.14285714
  80.          82.85714286  85.71428571  88.57142857  91.42857143
  94.28571429  97.14285714 100.         102.85714286 105.71428571
 108.57142857 111.42857143 114.28571429 117.14285714 120.        ]


In [ ]:
r_many = interp(tgt_many, z_lazy, fz_lazy)
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    %time r_many.compute()
visualize([prof, rprof, cprof])

In [5]:
%time interp(tgt_many, z, fz)
None

CPU times: user 7.34 s, sys: 311 ms, total: 7.65 s
Wall time: 7.65 s


In [6]:
%time interp(tgt_many, z_lazy, fz_lazy).compute()
None

CPU times: user 17.9 s, sys: 5.26 s, total: 23.2 s
Wall time: 7.14 s


In [ ]:
r_single = interp(tgt_single, z_lazy, fz_lazy)
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
    %time r_single.compute()
visualize([prof, rprof, cprof])

In [13]:
%time interp(tgt_single, z, fz)
None

CPU times: user 1.4 s, sys: 74.4 ms, total: 1.48 s
Wall time: 1.48 s


In [14]:
%time interp(tgt_single, z_lazy, fz_lazy).compute()
None

CPU times: user 15 s, sys: 20.4 s, total: 35.4 s
Wall time: 13.2 s


In [ ]:
%%time
for i in range(0, 3000, 750):
    for j in range(0, 1000, 500):
        interp(tgt_single, z[i:i+750, j:j+500], fz[i:i+750, j:j+500])